Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Dubov Andrey"
COLLABORATORS = ""

---

In [2]:
import numpy as np

from numpy.testing import assert_allclose


### Часть I. Постройте отражение Хаусхолдера для вектора.

Дан вектор $\mathbf{x}$ и плоскость, заданная вектором нормали $\mathbf{u}$. Преобразование Хаусхолдера отражает $\mathbf{x}$ относительно плоскости.

Матрица преобразований Хаусхолдера:
$$ \mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T $$

Если даны два вектора $\mathbf{x}$ и $\mathbf{y}$ одинаковой длины, поворот, преобразующий $\mathbf{x}$ в $\mathbf{y}$ называется преобразованием Хаусхолдера с
$$ \mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|} $$

Напишите функцию, преобразующую заданный вектор $\mathbf{x} = (x_1, \dots, x_n)$ в $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$, используя преобразование Хаусхолдера.


In [3]:
def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    u= vec.copy()
    u[0] = (-vec[1:]**2).sum()/(vec[0] + np.linalg.norm(vec, ord = 2))
    u /=np.linalg.norm(u, ord = 2)
    u = u.reshape(vec.shape[0], 1)
    H = np.eye(vec.shape[0]) - 2 * u * u.T

    
    return np.dot(H, vec), H

Протестируйте свою функцию на следующих примерах:

In [4]:
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(h @ v1, v, atol=1e-14)
assert_allclose(h @ v, v1, atol=1e-14)

assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(3), atol=1e-14)

###BEGIN HIDDEN TESTS
rndm = np.random.RandomState(1223)
vec = rndm.uniform(size=21)
v1, h = householder(vec)

assert_allclose(h @ v1, vec, atol=1e-14)
assert_allclose(h @ vec, v1, atol=1e-14)
assert_allclose(v1[1:], 0, atol=1e-14)

assert_allclose(h @ h.T, np.eye(h.shape[0]), atol=1e-14)
###END HIDDEN TESTS

### Part II. Вычисление $\mathrm{QR}$ - разложения матрицы.

Дана прямоугольная $m\times n$ матрица $\mathbf{A}$. Выполните отражение Хаусхолдера матрицы $\mathbf{H}_1$, преобразующее первый столбец матрицы $\mathbf{A}$ (назовем результат $\mathbf{A}^{(1)}$)

$$ 
\mathbf{H}_1 \mathbf{A} %\begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(1)}\;. 
$$

Теперь рассмотрим нижнюю правую часть матрицы $\mathbf{A}^{(1)}$ и выполним преобразование Хаусхолдера, действующее на 2 столбец $\mathbf{A}$:
$$
\mathbf{H}_2 \mathbf{A}^{(1)} % \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}
%
\equiv \mathbf{A}^{(2)} \;. $$

Повторяя процесс $n$ раз, получим
$$
\mathbf{H}_{n} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;, 
$$

где $\mathbf{R}$ верхняя треугольная матрица. Так как каждая из матриц $\mathbf{H}_k$ ортогональна, таковым будет и их произведение. Обратная от ортогональной также есть матрица ортогональная. Таким образом, алгоритм создает $\mathrm{QR}$ - разложение матрицы $\mathbf{A}$.

Напишите функцию, получающую прямоугольную матрицу $A$ и возвращающую матрицы $Q$ и $R$ --- компоненты $QR$-разложения $A$.


In [5]:
def qr_decomp(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    Q = np.eye(m)
    
    for i in range(1, n+1):
        
        #Первый вектор подматрицы
        vect = a1[i-1:, i-1]
        y_i, H_i = householder(vect)
        #Матрицу Хаусхолдера мы получаем, каждый раз итерируя по подматрице, т.е. ее размер уменьшается
        #Следовательно, для умножения Q @ H_i, необходимо привести ее к размеру матрицы Q
        I = np.eye(m)
        I[i-1:,i-1:] = H_i
        H_i = I 
    
        a1 = H_i @ a1
        Q = Q @ H_i

    return Q, a1

In [6]:
# можете запустить данную операцию для бооее сжатого вывода: нули вместо `1e-16` и т.д.
np.set_printoptions(suppress=True)

In [7]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
aa = a.copy()

q, r = qr_decomp(a)

# check that `qr_decomp` leaves `a` intact
assert_allclose(a, aa, atol=1e-16)

# тестируем, что Q ортогональна
assert_allclose(q @ q.T, np.eye(5), atol=1e-14)

# проверяем разложение
assert_allclose(q @ r, a, atol=1e-14)

Теперь сравните ваше разложение с разложением, полученным библиотечной функцией (содержащей соответствующие функции библиотеки LAPACK)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a,
                atol=1e-14)

In [9]:
#Создадим матрицу А
rndm = np.random.RandomState(1200)
a = rndm.uniform(size=(5, 3))
aa = a.copy()
#Воспользуемся нашей функцией
q, r = qr_decomp(a)
print("\nМатрицы A, Q, R:")
print(a,q,r,sep="\n\n")
print("\nПроверим матрицу Q на ортогональность:")
print(q @ q.T)
print("\nПеремножим матрицы Q и R:")
print(q @ r)
print("\nПроверим ответ:")
print(q @ r - a)

#Встроенная функция разложения
print("\nТесты встроенной функции:")
qq, rr = qr(a)
print("\nМатрицы Q, R:")
print(qq,rr,sep="\n\n", end="\n\n")
print("Проверим, совпадают ли матрицы Q")
print(qq)
print(q)
print("\n\nПроверим, совпадают ли матрицы R")
print(rr)
print(r)


Матрицы A, Q, R:
[[0.56195368 0.96236795 0.67366235]
 [0.85555573 0.07805584 0.35249985]
 [0.14863125 0.2651789  0.48787821]
 [0.98144582 0.4975699  0.18191398]
 [0.34072437 0.55728843 0.83726815]]

[[ 0.38332164  0.71420562 -0.25332605  0.21651491  0.48158286]
 [ 0.58359441 -0.52127192  0.39855654  0.26124328  0.40074653]
 [ 0.10138482  0.20124181  0.43625008 -0.84070541  0.22830492]
 [ 0.66946696 -0.12478735 -0.47660843 -0.31992726 -0.45467908]
 [ 0.232416    0.40263601  0.59959059  0.27519849 -0.59044387]]

[[ 1.46601084  0.90396388  0.82978965]
 [ 0.          0.86229933  0.7099804 ]
 [-0.         -0.          0.59798817]
 [-0.         -0.          0.        ]
 [-0.         -0.          0.        ]]

Проверим матрицу Q на ортогональность:
[[ 1.  0.  0.  0.  0.]
 [ 0.  1. -0. -0. -0.]
 [ 0. -0.  1.  0.  0.]
 [ 0. -0.  0.  1. -0.]
 [ 0. -0.  0. -0.  1.]]

Перемножим матрицы Q и R:
[[0.56195368 0.96236795 0.67366235]
 [0.85555573 0.07805584 0.35249985]
 [0.14863125 0.2651789  0.487878

Проверьте, согласуются ли ваши матрицы `q` и `r` с `qq` и `rr`. Объясните результат.
Оставьте пояснения в этой ячейке.


### Часть III. Безматричная реализация.

Отметим необычную структуру матрицы Хаусхолдера: матрица поворота $\mathbf{H}$ полностью характеризуется вектором отражения $\mathbf{u}$. Заметим, также, что вычислительная сложность операции отражения матрицы сильно зависит от порядка операций:
$$ \left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A} \qquad \textrm{is } O(m^2 n)\;, $$

тогда как $$ \mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn) $$

Таким образом, следует избегать формирований матриц $\mathbf{H}$. Вместо этого можно сохранять сами векторы отражения $\mathbf{u}$ и производить умножение произвольной матрицы на $\mathbf{Q}^T$, например, как отдельную функцию (класс).

Напишите функцию, выполняющую QR - разложение матрицы без формирования матриц $\mathbf{H}$ и возвращающую матрицу $\mathbf{R}$, а также вектора отражений Хаусхолдера.


Напишите вторую функцию, которая использует вектора отражений, полученных предыдущей функцией, для вычисления произведения $Q^T B$ для заданной матрицы B. Убедитесь, что вы добавили достаточно комментариев, следующих вашим выкладкам. 



In [10]:
def qr_nomatrix(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    #U - сюда будем складывать векторы u
    U = np.eye(m)
    for i in range(n):
        #Берем первый столбец подматрицы
        vec = np.transpose(a1)[i:, i:][0]
        y = np.zeros_like(vec)
        y[0] = np.linalg.norm(vec, 2)
        #Извлекаем вектор u
        u = (vec - y)/ np.linalg.norm(vec - y, 2)
        u = u.reshape(len(vec), 1) 
        
        #Преобразуем столбец подматрицы
        a1[i: , i: ] =  a1[i: , i: ] - 2 * u @ (np.transpose(u) @ a1[i: , i: ])
        
        #Добавим вектор u в матрицу U
        U[i:, i] = u.reshape(m-i)
        
    return a1, U

    
def householder_apply(b, uu):
    a1 = np.array(b, copy=True, dtype=float)
    m, n = a1.shape
    a2 = np.transpose(a1)
    
    
    #Так как умножение Q* на B в результате дает матрицу B', чьи столбцы состоят из векторов Q* @ b, 
    #то нам достаточно понять как умножить Q* на вектор
    B = np.zeros_like(a1)
    for i in range(n):
        b1 = a2[i]
        for j in range(m):
            u = uu[j:, j].reshape(m - j, 1)
            #Алгоритм очень похож на предыдущий, но используются вектора
            b1[j:] = b1[j:] - 2 * u @ (u.T @ b1[j:])
        B[i] = b1
    return B.T

In [11]:
rndm = np.random.RandomState(1223)

a = rndm.uniform(size=(5, 5))
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
R_lib = qr(a)[1]
assert_allclose(R1, R2, atol=1e-14)

###BEGIN HIDDEN TESTS

name2 = "Test 2: vertical rectangular matrix"
a2 = rndm.uniform(size=(5, 3))
R1, U = qr_nomatrix(a2)
R22 = householder_apply(a2, U)
R_lib2 = qr(a2)[1]
assert_allclose(R1, R22, atol=1e-14)


name3 = "Test 3: horizontal rectangular matrix"
a3 = rndm.uniform(size=(3, 5))
R1, U = qr_nomatrix(a3)
R23 = householder_apply(a3, U)
R_lib3 = qr(a3)[1]
assert_allclose(R1, R23, atol=1e-14)

###END HIDDEN TESTS

ValueError: could not broadcast input array from shape (5) into shape (3)

In [12]:
# Check consistency with the scipy library decomposition. Allow for sign differences

conds = [np.allclose(R2[i, :], R_lib[i, :], atol=1e-14) or
         np.allclose(R2[i, :], -R_lib[i, :], atol=1e-14) for i in range(5)]
assert False not in conds

###BEGIN HIDDEN TESTS
conds = [np.allclose(R22[i, :], R_lib2[i, :], atol=1e-14) or
         np.allclose(R22[i, :], -R_lib2[i, :], atol=1e-14) for i in range(5)]
assert False not in conds

conds = [np.allclose(R23[i, :], R_lib3[i, :], atol=1e-14) or
         np.allclose(R23[i, :], -R_lib3[i, :], atol=1e-14) for i in range(3)]
assert False not in conds
###END HIDDEN TESTS

NameError: name 'R22' is not defined

In [13]:
# More testing here, keep this cell intact

### BEGIN HIDDEN TESTS

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))

q, r = qr_decomp(a)
rr, uu = qr_nomatrix(a)

# check consistency
assert_allclose(np.triu(r),
                np.triu(rr), atol=1e-14)

# $Q^T @ A = R$
assert_allclose(householder_apply(a, uu),
                r,
                atol=1e-14)

# act on a different matrix
b = rndm.uniform(size=(5, 3))
assert_allclose(householder_apply(b, uu),
                q.T @ b,
                atol=1e-14)

# END HIDDEN TESTS

ValueError: could not broadcast input array from shape (5) into shape (3)